In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks as cf

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

## Imports
Import modules utilised, specifically **Pandas** for data handling and **Plotly** for interactive data visualisation. 

I also have defined options within Plotly that allow for the graphs to be saved and manipulated locally for portability.

In [2]:
df = pd.read_csv("Tesla-BusinessProductivity.csv")

In [3]:
df.shape

(23, 3)

## Data Shape
Showing 23 instances of of data in each dataset, of which there are 3 in total.

In [4]:
df.describe()

,Tesla Sales,Real Output per Hour
count,23.000000,23.000000
mean,13925.130435,106.461478
std,9136.266914,1.573489
min,321.000000,104.059000
25%,6674.500000,105.108500
50%,11507.000000,106.668000
75%,22100.000000,107.523500
max,29980.000000,109.050000


## Quick Descriptive Statistics
No need to define these variables further, as they're simplistic enough:
* Few in number, no need to sample
* Scalar in values, not categorical or ordinal

Even though we don't need to sample, best to remember that this is still assumed to be a representative sample of the true distributions of each individual dataset.

In [5]:
df.head()

,Date,Tesla Sales,Real Output per Hour
0,2012Q3,321,104.503
1,2012Q4,2400,104.059
2,2013Q1,4900,104.304
3,2013Q2,5150,104.147
4,2013Q3,5500,104.570


In [6]:
df.tail()

,Date,Tesla Sales,Real Output per Hour
18,2017Q1,25000,107.683
19,2017Q2,22000,108.142
20,2017Q3,26150,108.849
21,2017Q4,29870,108.943
22,2018Q1,29980,109.050


In [25]:
y0 = df['Tesla Sales']
y1 = df['Real Output per Hour']

trace0 = go.Box(
    y=y0,
    name = 'Tesla Sales',
    marker = dict(
        color = 'rgb(214, 12, 140)',
        )
    )
trace1 = go.Box(
    y=y1,
    name = 'Real Output per Hour',
    marker = dict(
        color = 'rgb(0, 128, 128)',
        )
    )

data = [trace0, trace1]
iplot(data)

In [9]:
iplot([go.Scatter(x=df['Date'], y=df['Tesla Sales'])])

In [10]:
iplot([go.Scatter(x=df['Date'], y=df['Real Output per Hour'])])

In [13]:
df.tail()

,Date,Tesla Sales,Real Output per Hour
18,2017Q1,25000,107.683
19,2017Q2,22000,108.142
20,2017Q3,26150,108.849
21,2017Q4,29870,108.943
22,2018Q1,29980,109.050


In [23]:
fig0 = df.iplot(columns=['Tesla Sales'], asFigure=True, kind='box')
fig1 = df.iplot(columns=['Real Output per Hour'], kind='box', secondary_y=['Real Output per Hour'], asFigure=True)
fig1['data'].extend(fig0['data'])
iplot(fig1)